In [1]:
import pandas as pd
import os
from datetime import datetime

[https://analisi.transparenciacatalunya.cat/Salut/Impacte-del-COVID-19-per-estat-de-vacunaci-/6izj-g3sb](https://analisi.transparenciacatalunya.cat/Salut/Impacte-del-COVID-19-per-estat-de-vacunaci-/6izj-g3sb)

Aquestes dades provenen de diversos sistemes d'informació del Departament de Salut i del Servei Català de la Salut, detallats a https://dadescovid.cat/documentacio , i mostren, per a la data de referencia de cada registre, per sexe, per grup d'edat i per a tot Catalunya, des de l'inici de la campanya de vacunació, l'1 de gener de 2021, l'impacte del COVID-19 (nous casos, hospitalitzacions, hospitalitzacions en crítics i defuncions) segons l'estat de vacunació de les persones afectades.

Cal tenir en compte que una mateixa persona està comptada tantes vegades com esdeveniments pels quals hagi passat.

### Cal tenir en compte que una mateixa persona està comptada tantes vegades com esdeveniments pels quals hagi passat.

### Usem els casos COVID-19 confirmats per PCR/TAR per al seguiment epidemiològic
Primer de tot, els casos de COVID-19 confirmats per fer un seguiment epidemiològic són exclusivament els confirmats per PCR/TAR, ja que la PCR/TAR+ ens indica que és un cas actiu i pel seguiment epidemiològic és important la data en què s’inicia el cas.

### Hem de diferenciar el seguiment segons si la població és general o de residència
I en tercer lloc és important diferenciar dues poblacions, la general i la de residències, ja que el seu comportament epidemiològic és totalment diferent, com es va poder veure en la primera onada. Per una banda, la població general viu en la comunitat i es relaciona i mou d’una determinada manera; les persones ingressades en residències geriàtriques, viuen en comunitats molt tancades i en llocs molt tancats i la seva mobilitat és molt reduïda. Donades aquestes característiques, no es pot fer el seguiment epidemiològic conjunt de les dues poblacions i és per això que a la web presentem les dades separades.

In [2]:
# handlig dates
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

In [3]:
filepath = 'C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/dades/impacte_covid_vacunacio.csv'

In [4]:
df = pd.read_csv(filepath,parse_dates=['DATA'], date_parser=dateparse)
df.head(5)

,SEXE,EDAT,DATA,ESDEVENIMENT,PAUTA,RECOMPTE
0,Dona,0 a 9,2022-01-12,Cas,Parcial,107
1,Dona,0 a 9,2022-01-12,Cas,No iniciada,784
2,Home,0 a 9,2022-01-12,Cas,No iniciada,865
3,Home,0 a 9,2022-01-12,Cas,Parcial,126
4,Home,0 a 9,2022-01-12,Hospitalització,No iniciada,2


In [5]:
df['ESDEVENIMENT'].value_counts()

Cas                14764
Hospitalització     8030
Crítics             3566
Defunció            2423
Name: ESDEVENIMENT, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28783 entries, 0 to 28782
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   SEXE          28783 non-null  object        
 1   EDAT          28783 non-null  object        
 2   DATA          28783 non-null  datetime64[ns]
 3   ESDEVENIMENT  28783 non-null  object        
 4   PAUTA         28783 non-null  object        
 5   RECOMPTE      28783 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.3+ MB


In [7]:
df.shape

(28783, 6)

In [8]:
# sum homes and dones
df_unisex = df.groupby([ "DATA", "EDAT", "ESDEVENIMENT", "PAUTA"])[["RECOMPTE"]].sum() 

In [9]:
df_unisex.head(25)

RECOMPTE
DATA       EDAT     ESDEVENIMENT    PAUTA                
2021-01-01 0 a 9    Cas             No iniciada        57
           10 a 19  Cas             No iniciada       115
           20 a 29  Cas             No iniciada       157
                    Hospitalització No iniciada         2
           30 a 39  Cas             No iniciada       184
                    Crítics         No iniciada         1
                    Hospitalització No iniciada         7
           40 a 49  Cas             No iniciada       237
                    Crítics         No iniciada         1
                    Hospitalització No iniciada        15
           50 a 59  Cas             No iniciada       231
                    Crítics         No iniciada         3
                    Defunció        No iniciada         1
                    Hospitalització No iniciada        29
           60 a 69  Cas             No iniciada       119
                    Crítics         No iniciada         6
                    Defunció        No iniciada         7
                    Hospitalització No iniciada        29
           70 a 79  Cas             No iniciada        66
                    Crítics         No iniciada         4
                    Defunció        No iniciada         8
                    Hospitalització No iniciada        27
           80 o més Cas             No iniciada        94
                    Crítics         No iniciada         2
                    Defunció        No iniciada        33

Usarem els casos registrats a Catalunya entre el 10 d'Octubre i el 10 de desembre on la variant delta era la majoritària.

Utilitzarem el [número de persones vacunades al 10 de novembre segons grup d'edat](https://analisi.transparenciacatalunya.cat/Salut/Vacunaci-per-al-COVID-19-dosis-administrades-per-r/tp23-dey4/data) i el [número total de persones per grups d'edat de l de gener](https://www.idescat.cat/pub/?id=aec&n=253) per a normalitzar.

In [10]:
# Subset Pandas Dataframe Using Range of Dates
#df["begin_index_date" : "end_index_date]['2005-05-01':'2005-08-31']

df_unisex_targ_date = df_unisex['2021-10-10':'2021-12-10']
df_unisex_targ_date.head()

RECOMPTE
DATA       EDAT    ESDEVENIMENT PAUTA                
2021-10-10 0 a 9   Cas          No iniciada         6
           10 a 19 Cas          Completa            5
                                No iniciada         6
           20 a 29 Cas          Completa            3
                                No iniciada         4

In [11]:
df_unisex_targ_date = df_unisex_targ_date.reset_index()
df_unisex_targ_date.tail()

,DATA,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
2542,2021-12-10,80 o més,Defunció,No iniciada,3
2543,2021-12-10,80 o més,Hospitalització,Completa,33
2544,2021-12-10,80 o més,Hospitalització,No iniciada,5
2545,2021-12-10,No classificat,Cas,Completa,7
2546,2021-12-10,No classificat,Cas,No iniciada,126


### [Número de persones vacunades al 10 de novembre segons grup d'edat:](https://analisi.transparenciacatalunya.cat/Salut/Vacunaci-per-al-COVID-19-dosis-administrades-per-r/tp23-dey4/data)

In [12]:
path_vac = 'C:/capes_sql/vac_edats/vac_per_edats.csv'

In [13]:
df_vac = pd.read_csv(path_vac, parse_dates=['DATA'], date_parser=dateparse, index_col= ['DATA'])

In [14]:
df_vac.head(5)

,SEXE_CODI,SEXE,REGIO_CODI,REGIO,SECTOR_CODI,SECTOR,AGA_CODI,AGA,ABS_CODI,ABS,EDAT,DOSI,FABRICANT,NO_VACUNAT,RECOMPTE
DATA,,,,,,,,,,,,,,,
2021-03-24,0.0,Home,7803.0,BARCELONA CIUTAT,7851.0,BARCELONA EIXAMPLE,70.0,Barcelona Dreta,28.0,BARCELONA 2-H,75 a 79,1,Moderna / Lonza,NaN,2
2021-04-22,1.0,Dona,7802.0,METROPOLITANA NORD,7867.0,BARCELONÈS NORD I MARESME,33.0,Maresme Central,154.0,MATARÓ 3,50 a 54,2,BioNTech / Pfizer,NaN,1
2021-04-22,0.0,Home,7803.0,BARCELONA CIUTAT,7851.0,BARCELONA EIXAMPLE,70.0,Barcelona Dreta,31.0,BARCELONA 2-K,60 a 64,1,Oxford / AstraZeneca,NaN,4
2021-12-21,1.0,Dona,7802.0,METROPOLITANA NORD,7843.0,VALLÈS OCCIDENTAL OEST,66.0,Vallès Occidental Oest,363.0,SANT QUIRZE DEL VALLÈS,60 a 64,2,Moderna / Lonza,NaN,1
2021-04-23,1.0,Dona,7801.0,METROPOLITANA SUD,7831.0,BAIX LLOBREGAT NORD,65.0,Baix Llobregat Nord,360.0,MARTORELL RURAL,55 a 59,2,Moderna / Lonza,NaN,1


In [15]:
df_vac_grouped = df_vac.groupby([ "DATA",  "DOSI", "EDAT"])[["RECOMPTE"]].sum() 

In [16]:
df_vac_grouped.head()

RECOMPTE
DATA       DOSI EDAT             
2020-12-27 1    15 a 19         1
                20 a 24        21
                25 a 29        13
                30 a 34        19
                35 a 39        23

In [17]:
# Check min value of index 
print(df_vac.index.min())

# Check max value of index 
print(df_vac.index.max())

2020-12-27 00:00:00
2022-01-13 00:00:00


In [18]:
df_vac_dates = df_vac_grouped['2020-12-27':'2021-11-10'] # 10 d'Octubre

In [19]:
df_vac_dates.tail(30)

RECOMPTE
DATA       DOSI EDAT                    
2021-11-10 2    20 a 24              293
                25 a 29              299
                30 a 34              270
                35 a 39              176
                40 a 44              165
                45 a 49              105
                50 a 54               85
                55 a 59               78
                60 a 64               74
                65 a 69               44
                70 a 74              610
                75 a 79               82
                80 o més              39
                No classificat        35
           3    12 a 14                1
                15 a 19                2
                20 a 24                3
                25 a 29                9
                30 a 34               15
                35 a 39               11
                40 a 44               27
                45 a 49               26
                50 a 54               37
                55 a 59               52
                60 a 64               62
                65 a 69              126
                70 a 74             3668
                75 a 79             8294
                80 o més           12914
                No classificat        22

Els grups de població per edats donades les dades seran:


0,19

20,29

30,39

40,49

50,59

60,69

70,79

80 o més

### Per tant anem a fer les agrupacions per edat correctes pels dos datasets.

In [20]:
df_unisex_targ_date.head(6)

,DATA,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,2021-10-10,0 a 9,Cas,No iniciada,6
1,2021-10-10,10 a 19,Cas,Completa,5
2,2021-10-10,10 a 19,Cas,No iniciada,6
3,2021-10-10,20 a 29,Cas,Completa,3
4,2021-10-10,20 a 29,Cas,No iniciada,4
5,2021-10-10,20 a 29,Hospitalització,No iniciada,2


In [21]:
#gettting rows on a specific condition
df_unisex_targ_date.loc[df_unisex_targ_date['EDAT'] == "0 a 9"]

,DATA,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,2021-10-10,0 a 9,Cas,No iniciada,6
27,2021-10-11,0 a 9,Cas,No iniciada,37
28,2021-10-11,0 a 9,Hospitalització,No iniciada,1
61,2021-10-12,0 a 9,Cas,No iniciada,17
88,2021-10-13,0 a 9,Cas,No iniciada,30
...,...,...,...,...,...
2331,2021-12-07,0 a 9,Hospitalització,No iniciada,2
2386,2021-12-08,0 a 9,Cas,No iniciada,245
2437,2021-12-09,0 a 9,Cas,No iniciada,685
2438,2021-12-09,0 a 9,Hospitalització,No iniciada,2


In [22]:
#gettting rows on a specific condition
df_unisex_targ_date.loc[df_unisex_targ_date['EDAT']  == "10 a 19"]

,DATA,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
1,2021-10-10,10 a 19,Cas,Completa,5
2,2021-10-10,10 a 19,Cas,No iniciada,6
29,2021-10-11,10 a 19,Cas,Completa,1
30,2021-10-11,10 a 19,Cas,No iniciada,30
62,2021-10-12,10 a 19,Cas,Completa,5
...,...,...,...,...,...
2442,2021-12-09,10 a 19,Hospitalització,No iniciada,1
2495,2021-12-10,10 a 19,Cas,Completa,183
2496,2021-12-10,10 a 19,Cas,No iniciada,424
2497,2021-12-10,10 a 19,Cas,Parcial,8


### Volem canviar els rangs d'edat de 0 a 9 i de 10 a 19 a 0 fins a 19 i sumar els camps iguals

What is regex true in pandas?

Regular expressions (regex) are essentially text patterns that you can use to automate searching through and replacing elements within strings of text

In [23]:
df_unisex_targ_date['EDAT'] = df_unisex_targ_date['EDAT'].str.replace(r'(0 a 9|10 a 19)', '0 a 19', regex=True)

In [24]:
df_unisex_targ_date

,DATA,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,2021-10-10,0 a 19,Cas,No iniciada,6
1,2021-10-10,0 a 19,Cas,Completa,5
2,2021-10-10,0 a 19,Cas,No iniciada,6
3,2021-10-10,20 a 29,Cas,Completa,3
4,2021-10-10,20 a 29,Cas,No iniciada,4
...,...,...,...,...,...
2542,2021-12-10,80 o més,Defunció,No iniciada,3
2543,2021-12-10,80 o més,Hospitalització,Completa,33
2544,2021-12-10,80 o més,Hospitalització,No iniciada,5
2545,2021-12-10,No classificat,Cas,Completa,7


In [25]:
# grupby for summing the equals fields
df_unisex_date_age_prov = df_unisex_targ_date.groupby([ "DATA", "EDAT", "ESDEVENIMENT", "PAUTA"])[["RECOMPTE"]].sum() 

In [26]:
df_unisex_date_age = df_unisex_date_age_prov.reset_index()
df_unisex_date_age

,DATA,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,2021-10-10,0 a 19,Cas,Completa,5
1,2021-10-10,0 a 19,Cas,No iniciada,12
2,2021-10-10,20 a 29,Cas,Completa,3
3,2021-10-10,20 a 29,Cas,No iniciada,4
4,2021-10-10,20 a 29,Hospitalització,No iniciada,2
...,...,...,...,...,...
2473,2021-12-10,80 o més,Defunció,No iniciada,3
2474,2021-12-10,80 o més,Hospitalització,Completa,33
2475,2021-12-10,80 o més,Hospitalització,No iniciada,5
2476,2021-12-10,No classificat,Cas,Completa,7


 Next we will procede changing the age ranges to the vaccination dataset.

In [27]:
df_vac_dates.tail(20)

RECOMPTE
DATA       DOSI EDAT                    
2021-11-10 2    70 a 74              610
                75 a 79               82
                80 o més              39
                No classificat        35
           3    12 a 14                1
                15 a 19                2
                20 a 24                3
                25 a 29                9
                30 a 34               15
                35 a 39               11
                40 a 44               27
                45 a 49               26
                50 a 54               37
                55 a 59               52
                60 a 64               62
                65 a 69              126
                70 a 74             3668
                75 a 79             8294
                80 o més           12914
                No classificat        22

In [28]:
df_vac_dates_prov = df_vac_dates.reset_index()
df_vac_dates_prov.tail(5)

,DATA,DOSI,EDAT,RECOMPTE
10544,2021-11-10,3,65 a 69,126
10545,2021-11-10,3,70 a 74,3668
10546,2021-11-10,3,75 a 79,8294
10547,2021-11-10,3,80 o més,12914
10548,2021-11-10,3,No classificat,22


In [29]:
df_vac_dates_edat_prov = df_vac_dates_prov.groupby([ "EDAT", "DOSI"])[["RECOMPTE"]].sum().reset_index()

In [30]:
df_vac_dates_edat_prov.head(6)

,EDAT,DOSI,RECOMPTE
0,0 a 11,1,381
1,0 a 11,2,152
2,0 a 11,3,1
3,12 a 14,1,217751
4,12 a 14,2,181507
5,12 a 14,3,276


In [31]:
# redoing the range of ages
df_vac_dates_edat_prov['EDAT'] = df_vac_dates_edat_prov['EDAT'].str.replace(r'(0 a 11|12 a 14|15 a 19)', '0 a 19', regex=True)
df_vac_dates_edat_prov['EDAT'] = df_vac_dates_edat_prov['EDAT'].str.replace(r'(20 a 24|25 a 29)', '20 a 29', regex=True)
df_vac_dates_edat_prov['EDAT'] = df_vac_dates_edat_prov['EDAT'].str.replace(r'(30 a 34|35 a 39)', '30 a 39', regex=True)
df_vac_dates_edat_prov['EDAT'] = df_vac_dates_edat_prov['EDAT'].str.replace(r'(40 a 44|45 a 49)', '40 a 49', regex=True)
df_vac_dates_edat_prov['EDAT'] = df_vac_dates_edat_prov['EDAT'].str.replace(r'(50 a 54|55 a 59)', '50 a 59', regex=True)
df_vac_dates_edat_prov['EDAT'] = df_vac_dates_edat_prov['EDAT'].str.replace(r'(60 a 64|65 a 69)', '60 a 69', regex=True)
df_vac_dates_edat_prov['EDAT'] = df_vac_dates_edat_prov['EDAT'].str.replace(r'(70 a 74|75 a 79)', '70 a 79', regex=True)

In [32]:
df_vac_dates_edat_prov.head(6)                                                                    

,EDAT,DOSI,RECOMPTE
0,0 a 19,1,381
1,0 a 19,2,152
2,0 a 19,3,1
3,0 a 19,1,217751
4,0 a 19,2,181507
5,0 a 19,3,276


In [33]:
df_vac_dates_edat = df_vac_dates_edat_prov.groupby(["EDAT", "DOSI"])[["RECOMPTE"]].sum().reset_index()

In [34]:
df_vac_dates_edat

,EDAT,DOSI,RECOMPTE
0,0 a 19,1,571012
1,0 a 19,2,466410
2,0 a 19,3,950
3,20 a 29,1,647869
4,20 a 29,2,508082
5,20 a 29,3,1876
6,30 a 39,1,803139
7,30 a 39,2,666998
8,30 a 39,3,3474
9,40 a 49,1,1152651


### Per a poder normalitzar només falta la població total:

In [35]:
path_edats = 'C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/dades/edats.csv'

In [36]:
df_edat_prov =  pd.read_csv(path_edats, thousands='.')

In [37]:
df_edat_prov

,Unnamed: 0,Homes,Dones,Total
0,De 0 a 4 anys,169716,161313,331029
1,De 5 a 9 anys,202834,190777,393611
2,De 10 a 14 anys,224622,21002,434642
3,De 15 a 19 anys,214844,197232,412076
4,De 20 a 24 anys,210054,194782,404836
5,De 25 a 29 anys,222311,215356,437667
6,De 30 a 34 anys,231914,233617,465531
7,De 35 a 39 anys,260651,265245,525896
8,De 40 a 44 anys,325347,320002,645349
9,De 45 a 49 anys,330936,319863,650799


In [38]:
# rename col
df_edat_prov.rename(columns={'Unnamed: 0': 'EDAT'}, inplace=True)

In [39]:
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 0 a 4 anys|De 5 a 9 anys|De 10 a 14 anys|De 15 a 19 anys)', '0 a 19', regex=True)
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 20 a 24 anys|De 25 a 29 anys)', '20 a 29', regex=True)
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 30 a 34 anys|De 35 a 39 anys)', '30 a 39', regex=True)
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 40 a 44 anys|De 45 a 49 anys)', '40 a 49', regex=True)
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 50 a 54 anys|De 55 a 59 anys)', '50 a 59', regex=True)
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 60 a 64 anys|De 65 a 69 anys)', '60 a 69', regex=True)
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 70 a 74 anys|De 75 a 79 anys)', '70 a 79', regex=True)
df_edat_prov['EDAT'] = df_edat_prov['EDAT'].str.replace(r'(De 80 a 84 anys|De 85 a 89 anys|De 90 a 94 anys|De 95 anys i més)', '80 o més', regex=True)

In [40]:
df_edat_prov

,EDAT,Homes,Dones,Total
0,0 a 19,169716,161313,331029
1,0 a 19,202834,190777,393611
2,0 a 19,224622,21002,434642
3,0 a 19,214844,197232,412076
4,20 a 29,210054,194782,404836
5,20 a 29,222311,215356,437667
6,30 a 39,231914,233617,465531
7,30 a 39,260651,265245,525896
8,40 a 49,325347,320002,645349
9,40 a 49,330936,319863,650799


In [41]:
df_edat = df_edat_prov.groupby(["EDAT"])[["Total"]].sum().reset_index()

In [42]:
df_edat

,EDAT,Total
0,0 a 19,1571358
1,20 a 29,842503
2,30 a 39,991427
3,40 a 49,1296148
4,50 a 59,1104675
5,60 a 69,850137
6,70 a 79,632085
7,80 o més,451425


Arribats aquí ja tenim els tres datasets amb el rang d'edats corresponent.

Anem a calcular els percentatges de població vacunada per cada rang d'edats al 10 de Novembre.

In [43]:
df_vac_dates_edat.head()

,EDAT,DOSI,RECOMPTE
0,0 a 19,1,571012
1,0 a 19,2,466410
2,0 a 19,3,950
3,20 a 29,1,647869
4,20 a 29,2,508082


In [44]:
df_vac_dates_edat_m = pd.merge(df_vac_dates_edat, df_edat, how='left') # manté tots els registres del df de l'esquerra

In [45]:
df_vac_dates_edat_m.head()
# Total = pobl en el rang d'edat

,EDAT,DOSI,RECOMPTE,Total
0,0 a 19,1,571012,1571358.0
1,0 a 19,2,466410,1571358.0
2,0 a 19,3,950,1571358.0
3,20 a 29,1,647869,842503.0
4,20 a 29,2,508082,842503.0


In [46]:
df_vac_dates_edat_m['PERCENT'] = df_vac_dates_edat_m['RECOMPTE']/df_vac_dates_edat_m['Total']*100

In [47]:
df_vac_dates_edat_m.head()

,EDAT,DOSI,RECOMPTE,Total,PERCENT
0,0 a 19,1,571012,1571358.0,36.338759
1,0 a 19,2,466410,1571358.0,29.681969
2,0 a 19,3,950,1571358.0,0.060457
3,20 a 29,1,647869,842503.0,76.898124
4,20 a 29,2,508082,842503.0,60.306254


In [48]:
df_unisex_date_age

,DATA,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,2021-10-10,0 a 19,Cas,Completa,5
1,2021-10-10,0 a 19,Cas,No iniciada,12
2,2021-10-10,20 a 29,Cas,Completa,3
3,2021-10-10,20 a 29,Cas,No iniciada,4
4,2021-10-10,20 a 29,Hospitalització,No iniciada,2
...,...,...,...,...,...
2473,2021-12-10,80 o més,Defunció,No iniciada,3
2474,2021-12-10,80 o més,Hospitalització,Completa,33
2475,2021-12-10,80 o més,Hospitalització,No iniciada,5
2476,2021-12-10,No classificat,Cas,Completa,7


In [49]:
df_prova_casos = df_unisex_date_age.groupby(["EDAT", "ESDEVENIMENT", "PAUTA"])[["RECOMPTE"]].sum().reset_index()

In [50]:
df_prova_casos.head()

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,0 a 19,Cas,Completa,1912
1,0 a 19,Cas,No iniciada,16306
2,0 a 19,Cas,Parcial,83
3,0 a 19,Crítics,No iniciada,5
4,0 a 19,Defunció,No iniciada,1


No es tindra en compte l'afectació de la vacunació parcial ni terceres vacunes només la pauta completa de dues vacunes indenendentment del moment de ser administrades.

In [51]:
df_prova_casos_compl = df_prova_casos[df_prova_casos.PAUTA != 'Parcial']

In [52]:
df_prova_casos_compl.head()

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,0 a 19,Cas,Completa,1912
1,0 a 19,Cas,No iniciada,16306
3,0 a 19,Crítics,No iniciada,5
4,0 a 19,Defunció,No iniciada,1
5,0 a 19,Hospitalització,Completa,4


In [53]:
df_vac_dates_edat_m.head()

,EDAT,DOSI,RECOMPTE,Total,PERCENT
0,0 a 19,1,571012,1571358.0,36.338759
1,0 a 19,2,466410,1571358.0,29.681969
2,0 a 19,3,950,1571358.0,0.060457
3,20 a 29,1,647869,842503.0,76.898124
4,20 a 29,2,508082,842503.0,60.306254


In [54]:
df_vac_dates_edat__comp = df_vac_dates_edat_m[df_vac_dates_edat_m.DOSI == 2]

In [55]:
df_vac_dates_edat__comp

,EDAT,DOSI,RECOMPTE,Total,PERCENT
1,0 a 19,2,466410,1571358.0,29.681969
4,20 a 29,2,508082,842503.0,60.306254
7,30 a 39,2,666998,991427.0,67.276562
10,40 a 49,2,921497,1296148.0,71.095045
13,50 a 59,2,906191,1104675.0,82.032362
16,60 a 69,2,719363,850137.0,84.617303
19,70 a 79,2,582150,632085.0,92.099955
22,80 o més,2,427070,451425.0,94.604862
25,No classificat,2,25365,NaN,NaN


### 1. Com afecta la vacunació per a la prevenció de contagis?
### 2. Com afecta la vacunació per a la prevenció d'hospitalitzacions?
### 3. Com afecta la vacunació per a la prevenció d'ingressos a les UCIS?
### 4. Com afecta la vacunació per a la prevenció de defuncions?

## 1. Com afecta la vacunació per a la prevenció de contagis?

In [56]:
df_prova_casos_compl.head()

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,0 a 19,Cas,Completa,1912
1,0 a 19,Cas,No iniciada,16306
3,0 a 19,Crítics,No iniciada,5
4,0 a 19,Defunció,No iniciada,1
5,0 a 19,Hospitalització,Completa,4


In [57]:
# we choose only the number of cases from EDEVENIMENT
df_prova_casos_compl_casos = df_prova_casos_compl[df_prova_casos_compl.ESDEVENIMENT == 'Cas']

In [58]:
df_prova_casos_compl_casos

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,0 a 19,Cas,Completa,1912
1,0 a 19,Cas,No iniciada,16306
7,20 a 29,Cas,Completa,3976
8,20 a 29,Cas,No iniciada,1911
15,30 a 39,Cas,Completa,7074
16,30 a 39,Cas,No iniciada,2683
22,40 a 49,Cas,Completa,11866
23,40 a 49,Cas,No iniciada,2505
33,50 a 59,Cas,Completa,7381
34,50 a 59,Cas,No iniciada,1314


In [59]:
df_vac_dates_edat__comp

,EDAT,DOSI,RECOMPTE,Total,PERCENT
1,0 a 19,2,466410,1571358.0,29.681969
4,20 a 29,2,508082,842503.0,60.306254
7,30 a 39,2,666998,991427.0,67.276562
10,40 a 49,2,921497,1296148.0,71.095045
13,50 a 59,2,906191,1104675.0,82.032362
16,60 a 69,2,719363,850137.0,84.617303
19,70 a 79,2,582150,632085.0,92.099955
22,80 o més,2,427070,451425.0,94.604862
25,No classificat,2,25365,NaN,NaN


Anem a normalitzar-ho: 

In [60]:
df_prova_casos_norm = pd.merge(df_prova_casos_compl_casos, df_vac_dates_edat__comp[['EDAT', 'PERCENT', 'Total']], how='left') # manté tots els registres del df de l'esquerra

In [61]:
df_prova_casos_norm.head()

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total
0,0 a 19,Cas,Completa,1912,29.681969,1571358.0
1,0 a 19,Cas,No iniciada,16306,29.681969,1571358.0
2,20 a 29,Cas,Completa,3976,60.306254,842503.0
3,20 a 29,Cas,No iniciada,1911,60.306254,842503.0
4,30 a 39,Cas,Completa,7074,67.276562,991427.0


Percentatge de vacunats per grup d'edat.

In [62]:
# % * -1 + 100 = % no vac
df_prova_casos_norm.loc[df_prova_casos_norm.PAUTA == 'No iniciada', "PERCENT"] = df_prova_casos_norm['PERCENT']*-1+100

In [63]:
df_prova_casos_norm.head()

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total
0,0 a 19,Cas,Completa,1912,29.681969,1571358.0
1,0 a 19,Cas,No iniciada,16306,70.318031,1571358.0
2,20 a 29,Cas,Completa,3976,60.306254,842503.0
3,20 a 29,Cas,No iniciada,1911,39.693746,842503.0
4,30 a 39,Cas,Completa,7074,67.276562,991427.0


In [64]:
df_prova_casos_norm['TOTAL_GRUP'] = df_prova_casos_norm['Total']*(df_prova_casos_norm['PERCENT']/100)

In [65]:
df_prova_casos_norm.head()

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP
0,0 a 19,Cas,Completa,1912,29.681969,1571358.0,466410.0
1,0 a 19,Cas,No iniciada,16306,70.318031,1571358.0,1104948.0
2,20 a 29,Cas,Completa,3976,60.306254,842503.0,508082.0
3,20 a 29,Cas,No iniciada,1911,39.693746,842503.0,334421.0
4,30 a 39,Cas,Completa,7074,67.276562,991427.0,666998.0


In [66]:
df_prova_casos_norm["RISC"] = df_prova_casos_norm["RECOMPTE"]/df_prova_casos_norm["TOTAL_GRUP"]

In [67]:
df_prova_casos_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,0 a 19,Cas,Completa,1912,29.681969,1571358.0,466410.0,0.004099
1,0 a 19,Cas,No iniciada,16306,70.318031,1571358.0,1104948.0,0.014757
2,20 a 29,Cas,Completa,3976,60.306254,842503.0,508082.0,0.007826
3,20 a 29,Cas,No iniciada,1911,39.693746,842503.0,334421.0,0.005714
4,30 a 39,Cas,Completa,7074,67.276562,991427.0,666998.0,0.010606
5,30 a 39,Cas,No iniciada,2683,32.723438,991427.0,324429.0,0.008270
6,40 a 49,Cas,Completa,11866,71.095045,1296148.0,921497.0,0.012877
7,40 a 49,Cas,No iniciada,2505,28.904955,1296148.0,374651.0,0.006686
8,50 a 59,Cas,Completa,7381,82.032362,1104675.0,906191.0,0.008145
9,50 a 59,Cas,No iniciada,1314,17.967638,1104675.0,198484.0,0.006620


In [68]:
df_prova_casos_norm.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afect_vac_prevencio_contagis_brut.xlsx", sheet_name='Fulla1')

In [69]:
# select pair rows
df_prova_efect_completa = df_prova_casos_norm.iloc[::2]
df_prova_efect_completa

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,0 a 19,Cas,Completa,1912,29.681969,1571358.0,466410.0,0.004099
2,20 a 29,Cas,Completa,3976,60.306254,842503.0,508082.0,0.007826
4,30 a 39,Cas,Completa,7074,67.276562,991427.0,666998.0,0.010606
6,40 a 49,Cas,Completa,11866,71.095045,1296148.0,921497.0,0.012877
8,50 a 59,Cas,Completa,7381,82.032362,1104675.0,906191.0,0.008145
10,60 a 69,Cas,Completa,6731,84.617303,850137.0,719363.0,0.009357
12,70 a 79,Cas,Completa,3823,92.099955,632085.0,582150.0,0.006567
14,80 o més,Cas,Completa,2245,94.604862,451425.0,427070.0,0.005257
16,No classificat,Cas,Completa,118,NaN,NaN,NaN,NaN


In [70]:
# select odd rows
df_prova_efect_no_vac = df_prova_casos_norm.iloc[1:-2:2] # odd rows
df_prova_efect_no_vac

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
1,0 a 19,Cas,No iniciada,16306,70.318031,1571358.0,1104948.0,0.014757
3,20 a 29,Cas,No iniciada,1911,39.693746,842503.0,334421.0,0.005714
5,30 a 39,Cas,No iniciada,2683,32.723438,991427.0,324429.0,0.008270
7,40 a 49,Cas,No iniciada,2505,28.904955,1296148.0,374651.0,0.006686
9,50 a 59,Cas,No iniciada,1314,17.967638,1104675.0,198484.0,0.006620
11,60 a 69,Cas,No iniciada,802,15.382697,850137.0,130774.0,0.006133
13,70 a 79,Cas,No iniciada,293,7.900045,632085.0,49935.0,0.005868
15,80 o més,Cas,No iniciada,192,5.395138,451425.0,24355.0,0.007883


In [71]:
# delete non necessary fields 
df_calc_ef_completa = df_prova_efect_completa.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_calc_ef_completa

,EDAT,RISC
0,0 a 19,0.004099
2,20 a 29,0.007826
4,30 a 39,0.010606
6,40 a 49,0.012877
8,50 a 59,0.008145
10,60 a 69,0.009357
12,70 a 79,0.006567
14,80 o més,0.005257
16,No classificat,NaN


In [72]:
df_calc_ef_no_vac = df_prova_efect_no_vac.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_calc_ef_no_vac

,EDAT,RISC
1,0 a 19,0.014757
3,20 a 29,0.005714
5,30 a 39,0.008270
7,40 a 49,0.006686
9,50 a 59,0.006620
11,60 a 69,0.006133
13,70 a 79,0.005868
15,80 o més,0.007883


In [73]:
df_calc_ef_no_vac = df_calc_ef_no_vac.rename(columns={"RISC": "RISC_N_V",})

In [74]:
df_calc_m = df_calc_ef_no_vac.merge(df_calc_ef_completa, left_on='EDAT', right_on='EDAT')

In [75]:
df_calc_m

,EDAT,RISC_N_V,RISC
0,0 a 19,0.014757,0.004099
1,20 a 29,0.005714,0.007826
2,30 a 39,0.008270,0.010606
3,40 a 49,0.006686,0.012877
4,50 a 59,0.006620,0.008145
5,60 a 69,0.006133,0.009357
6,70 a 79,0.005868,0.006567
7,80 o més,0.007883,0.005257


In [76]:
# efectivitat = (risc placebo - risc vacunats) / risc placebo
df_calc_m['EFECTIVITAT'] = (df_calc_m['RISC_N_V'] - df_calc_m['RISC']) / df_calc_m['RISC_N_V'] * 100

In [77]:
df_calc_m

,EDAT,RISC_N_V,RISC,EFECTIVITAT
0,0 a 19,0.014757,0.004099,72.221139
1,20 a 29,0.005714,0.007826,-36.944760
2,30 a 39,0.008270,0.010606,-28.244727
3,40 a 49,0.006686,0.012877,-92.588160
4,50 a 59,0.006620,0.008145,-23.034129
5,60 a 69,0.006133,0.009357,-52.573298
6,70 a 79,0.005868,0.006567,-11.919775
7,80 o més,0.007883,0.005257,33.318680


In [78]:
df_calc_m.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afect_vac_prevenc_contagis.xlsx", sheet_name='Fulla1')

Veient només els resultats podriem pensar que la vacuna fins i tot és contraproduent en la prevenció de la infecció, però cal analitzar tots els factors.

a) En els estudis d'efectivitat publicats durant les fases d'investigació els no vacunats sels havia administrat placebo de manera que el comportament d'ambdós grups era molt més semblant al de la realitat actual on el comportament dels grups pot ser molt diferenciat no només en la prevenció de contagis sumant les restriccions que s'apliquen als no vacunats com sortir de festa o anar a restaurants sinó també amb la realització de pcrs que pot ser molt diferent en ambdós grup ja sigui per la percepció de risc o altres factors. 

b) Les dades si que mostren un augment de l'efectivitat en els grups d'edat de 0 a 19 anys i els de 80 o més. Aquests grups són els últims en ser vacunats, en dos dosis per els joves i el booster per els més grans.

Tot i les diferències de comportament que hi puguin haver entre grups si féssim un seguiment d'aquestes dades per als mesos següents és probable que mostressin una ràpida pèrdua d'efectivitat dels vaccins contra la infecció per Covid 19.

## 2. Com afecta la vacunació per a la prevenció d'hospitalitzacions?

In [79]:
df_prova_casos_compl

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,0 a 19,Cas,Completa,1912
1,0 a 19,Cas,No iniciada,16306
3,0 a 19,Crítics,No iniciada,5
4,0 a 19,Defunció,No iniciada,1
5,0 a 19,Hospitalització,Completa,4
...,...,...,...,...
75,80 o més,Hospitalització,No iniciada,78
77,No classificat,Cas,Completa,118
78,No classificat,Cas,No iniciada,1780
80,No classificat,Crítics,No iniciada,1


In [80]:
# we choose only the number of cases from EDEVENIMENT
df_hospi = df_prova_casos_compl[df_prova_casos_compl.ESDEVENIMENT == 'Hospitalització']

In [81]:
df_hospi

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
5,0 a 19,Hospitalització,Completa,4
6,0 a 19,Hospitalització,No iniciada,54
12,20 a 29,Hospitalització,Completa,15
13,20 a 29,Hospitalització,No iniciada,56
20,30 a 39,Hospitalització,Completa,47
21,30 a 39,Hospitalització,No iniciada,92
30,40 a 49,Hospitalització,Completa,71
31,40 a 49,Hospitalització,No iniciada,123
40,50 a 59,Hospitalització,Completa,141
41,50 a 59,Hospitalització,No iniciada,142


In [82]:
df_vac_dates_edat__comp

,EDAT,DOSI,RECOMPTE,Total,PERCENT
1,0 a 19,2,466410,1571358.0,29.681969
4,20 a 29,2,508082,842503.0,60.306254
7,30 a 39,2,666998,991427.0,67.276562
10,40 a 49,2,921497,1296148.0,71.095045
13,50 a 59,2,906191,1104675.0,82.032362
16,60 a 69,2,719363,850137.0,84.617303
19,70 a 79,2,582150,632085.0,92.099955
22,80 o més,2,427070,451425.0,94.604862
25,No classificat,2,25365,NaN,NaN


In [83]:
df_hospi_norm = pd.merge(df_hospi, df_vac_dates_edat__comp[['EDAT', 'PERCENT', 'Total']], how='left')

In [84]:
df_hospi_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total
0,0 a 19,Hospitalització,Completa,4,29.681969,1571358.0
1,0 a 19,Hospitalització,No iniciada,54,29.681969,1571358.0
2,20 a 29,Hospitalització,Completa,15,60.306254,842503.0
3,20 a 29,Hospitalització,No iniciada,56,60.306254,842503.0
4,30 a 39,Hospitalització,Completa,47,67.276562,991427.0
5,30 a 39,Hospitalització,No iniciada,92,67.276562,991427.0
6,40 a 49,Hospitalització,Completa,71,71.095045,1296148.0
7,40 a 49,Hospitalització,No iniciada,123,71.095045,1296148.0
8,50 a 59,Hospitalització,Completa,141,82.032362,1104675.0
9,50 a 59,Hospitalització,No iniciada,142,82.032362,1104675.0


In [85]:
# % * -1 + 100 = % no vac
df_hospi_norm.loc[df_hospi_norm.PAUTA == 'No iniciada', "PERCENT"] = df_hospi_norm['PERCENT']*-1+100

In [86]:
df_hospi_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total
0,0 a 19,Hospitalització,Completa,4,29.681969,1571358.0
1,0 a 19,Hospitalització,No iniciada,54,70.318031,1571358.0
2,20 a 29,Hospitalització,Completa,15,60.306254,842503.0
3,20 a 29,Hospitalització,No iniciada,56,39.693746,842503.0
4,30 a 39,Hospitalització,Completa,47,67.276562,991427.0
5,30 a 39,Hospitalització,No iniciada,92,32.723438,991427.0
6,40 a 49,Hospitalització,Completa,71,71.095045,1296148.0
7,40 a 49,Hospitalització,No iniciada,123,28.904955,1296148.0
8,50 a 59,Hospitalització,Completa,141,82.032362,1104675.0
9,50 a 59,Hospitalització,No iniciada,142,17.967638,1104675.0


In [87]:
df_hospi_norm['TOTAL_GRUP'] = df_hospi_norm['Total']*(df_hospi_norm['PERCENT']/100)

In [88]:
df_hospi_norm["RISC"] = df_hospi_norm["RECOMPTE"]/df_hospi_norm["TOTAL_GRUP"]

In [89]:
df_hospi_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,0 a 19,Hospitalització,Completa,4,29.681969,1571358.0,466410.0,0.000009
1,0 a 19,Hospitalització,No iniciada,54,70.318031,1571358.0,1104948.0,0.000049
2,20 a 29,Hospitalització,Completa,15,60.306254,842503.0,508082.0,0.000030
3,20 a 29,Hospitalització,No iniciada,56,39.693746,842503.0,334421.0,0.000167
4,30 a 39,Hospitalització,Completa,47,67.276562,991427.0,666998.0,0.000070
5,30 a 39,Hospitalització,No iniciada,92,32.723438,991427.0,324429.0,0.000284
6,40 a 49,Hospitalització,Completa,71,71.095045,1296148.0,921497.0,0.000077
7,40 a 49,Hospitalització,No iniciada,123,28.904955,1296148.0,374651.0,0.000328
8,50 a 59,Hospitalització,Completa,141,82.032362,1104675.0,906191.0,0.000156
9,50 a 59,Hospitalització,No iniciada,142,17.967638,1104675.0,198484.0,0.000715


In [90]:
df_hospi_norm.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afect_vac_hospi_brut.xlsx", sheet_name='Fulla1')

In [91]:
df_hospi_norm.head()

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,0 a 19,Hospitalització,Completa,4,29.681969,1571358.0,466410.0,0.000009
1,0 a 19,Hospitalització,No iniciada,54,70.318031,1571358.0,1104948.0,0.000049
2,20 a 29,Hospitalització,Completa,15,60.306254,842503.0,508082.0,0.000030
3,20 a 29,Hospitalització,No iniciada,56,39.693746,842503.0,334421.0,0.000167
4,30 a 39,Hospitalització,Completa,47,67.276562,991427.0,666998.0,0.000070


In [92]:
# select pair rows (vaccinació completa)
df_hospi_norm_completa = df_hospi_norm.iloc[::2]
df_hospi_norm_completa

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,0 a 19,Hospitalització,Completa,4,29.681969,1571358.0,466410.0,0.000009
2,20 a 29,Hospitalització,Completa,15,60.306254,842503.0,508082.0,0.000030
4,30 a 39,Hospitalització,Completa,47,67.276562,991427.0,666998.0,0.000070
6,40 a 49,Hospitalització,Completa,71,71.095045,1296148.0,921497.0,0.000077
8,50 a 59,Hospitalització,Completa,141,82.032362,1104675.0,906191.0,0.000156
10,60 a 69,Hospitalització,Completa,383,84.617303,850137.0,719363.0,0.000532
12,70 a 79,Hospitalització,Completa,463,92.099955,632085.0,582150.0,0.000795
14,80 o més,Hospitalització,Completa,572,94.604862,451425.0,427070.0,0.001339
16,No classificat,Hospitalització,No iniciada,17,NaN,NaN,NaN,NaN


In [93]:
# select odd rows
df_hospi_norm_no_vac = df_hospi_norm.iloc[1:-1:2] # odd rows
df_hospi_norm_no_vac

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
1,0 a 19,Hospitalització,No iniciada,54,70.318031,1571358.0,1104948.0,0.000049
3,20 a 29,Hospitalització,No iniciada,56,39.693746,842503.0,334421.0,0.000167
5,30 a 39,Hospitalització,No iniciada,92,32.723438,991427.0,324429.0,0.000284
7,40 a 49,Hospitalització,No iniciada,123,28.904955,1296148.0,374651.0,0.000328
9,50 a 59,Hospitalització,No iniciada,142,17.967638,1104675.0,198484.0,0.000715
11,60 a 69,Hospitalització,No iniciada,145,15.382697,850137.0,130774.0,0.001109
13,70 a 79,Hospitalització,No iniciada,88,7.900045,632085.0,49935.0,0.001762
15,80 o més,Hospitalització,No iniciada,78,5.395138,451425.0,24355.0,0.003203


In [94]:
# delete non necessary fields 
df_hospi_norm_completa_rf = df_hospi_norm_completa.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_hospi_norm_completa_rf

,EDAT,RISC
0,0 a 19,0.000009
2,20 a 29,0.000030
4,30 a 39,0.000070
6,40 a 49,0.000077
8,50 a 59,0.000156
10,60 a 69,0.000532
12,70 a 79,0.000795
14,80 o més,0.001339
16,No classificat,NaN


In [95]:
# delete non necessary fields 
df_hospi_norm_no_vac_rf = df_hospi_norm_no_vac.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_hospi_norm_no_vac_rf

,EDAT,RISC
1,0 a 19,0.000049
3,20 a 29,0.000167
5,30 a 39,0.000284
7,40 a 49,0.000328
9,50 a 59,0.000715
11,60 a 69,0.001109
13,70 a 79,0.001762
15,80 o més,0.003203


In [96]:
df_hospi_norm_no_vac_rf_ren = df_hospi_norm_no_vac_rf.rename(columns={"RISC": "RISC_N_V"})

In [97]:
df_hospi_norm_no_vac_rf_ren

,EDAT,RISC_N_V
1,0 a 19,0.000049
3,20 a 29,0.000167
5,30 a 39,0.000284
7,40 a 49,0.000328
9,50 a 59,0.000715
11,60 a 69,0.001109
13,70 a 79,0.001762
15,80 o més,0.003203


In [98]:
df_calc_hospi_m = df_hospi_norm_no_vac_rf_ren.merge(df_hospi_norm_completa_rf, left_on='EDAT', right_on='EDAT')

In [99]:
df_calc_hospi_m

,EDAT,RISC_N_V,RISC
0,0 a 19,0.000049,0.000009
1,20 a 29,0.000167,0.000030
2,30 a 39,0.000284,0.000070
3,40 a 49,0.000328,0.000077
4,50 a 59,0.000715,0.000156
5,60 a 69,0.001109,0.000532
6,70 a 79,0.001762,0.000795
7,80 o més,0.003203,0.001339


In [100]:
# efectivitat = (risc placebo - risc vacunats) / risc placebo
df_calc_hospi_m['EFECTIVITAT'] = (df_calc_hospi_m['RISC_N_V'] - df_calc_hospi_m['RISC']) / df_calc_hospi_m['RISC_N_V'] * 100

In [101]:
df_calc_hospi_m

,EDAT,RISC_N_V,RISC,EFECTIVITAT
0,0 a 19,0.000049,0.000009,82.451491
1,20 a 29,0.000167,0.000030,82.369568
2,30 a 39,0.000284,0.000070,75.151215
3,40 a 49,0.000328,0.000077,76.531453
4,50 a 59,0.000715,0.000156,78.251139
5,60 a 69,0.001109,0.000532,51.981998
6,70 a 79,0.001762,0.000795,54.869674
7,80 o més,0.003203,0.001339,58.179377


Sí veiem una clara efectivitat de la vacuna per reduïr les hospitalitzacions. I a diferència dels contagis on els grups poden comportar-se diferent davant dels símptomes i contactes positius o en l'exposició al virus en els casos on es necessari la hospitalització si és necessària és necessaria per ambdós grups i si no tens un test d'antígens positiu sel's aplicarà una PCR.

In [102]:
df_calc_hospi_m.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afectivitat_vacuna_hospi.xlsx", sheet_name='Fulla1')

## 3. Com afecta la vacunació per a la prevenció d'ingressos a les UCIS?

In [103]:
df_prova_casos_compl

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,0 a 19,Cas,Completa,1912
1,0 a 19,Cas,No iniciada,16306
3,0 a 19,Crítics,No iniciada,5
4,0 a 19,Defunció,No iniciada,1
5,0 a 19,Hospitalització,Completa,4
...,...,...,...,...
75,80 o més,Hospitalització,No iniciada,78
77,No classificat,Cas,Completa,118
78,No classificat,Cas,No iniciada,1780
80,No classificat,Crítics,No iniciada,1


In [104]:
# we choose only the number of cases Crítics from EDEVENIMENT
df_uci = df_prova_casos_compl[df_prova_casos_compl.ESDEVENIMENT == 'Crítics']

In [105]:
df_uci

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
3,0 a 19,Crítics,No iniciada,5
10,20 a 29,Crítics,Completa,1
11,20 a 29,Crítics,No iniciada,3
18,30 a 39,Crítics,Completa,5
19,30 a 39,Crítics,No iniciada,20
25,40 a 49,Crítics,Completa,17
26,40 a 49,Crítics,No iniciada,40
36,50 a 59,Crítics,Completa,29
37,50 a 59,Crítics,No iniciada,48
46,60 a 69,Crítics,Completa,88


In [106]:
df_vac_dates_edat__comp

,EDAT,DOSI,RECOMPTE,Total,PERCENT
1,0 a 19,2,466410,1571358.0,29.681969
4,20 a 29,2,508082,842503.0,60.306254
7,30 a 39,2,666998,991427.0,67.276562
10,40 a 49,2,921497,1296148.0,71.095045
13,50 a 59,2,906191,1104675.0,82.032362
16,60 a 69,2,719363,850137.0,84.617303
19,70 a 79,2,582150,632085.0,92.099955
22,80 o més,2,427070,451425.0,94.604862
25,No classificat,2,25365,NaN,NaN


In [107]:
df_uci_norm = pd.merge(df_uci, df_vac_dates_edat__comp[['EDAT', 'PERCENT', 'Total']], how='left')

In [108]:
# % * -1 + 100 = % no vac
df_uci_norm.loc[df_uci_norm.PAUTA == 'No iniciada', "PERCENT"] = df_uci_norm['PERCENT']*-1+100

In [109]:
df_uci_norm['TOTAL_GRUP'] = df_uci_norm['Total']*(df_uci_norm['PERCENT']/100)

In [110]:
df_uci_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP
0,0 a 19,Crítics,No iniciada,5,70.318031,1571358.0,1104948.0
1,20 a 29,Crítics,Completa,1,60.306254,842503.0,508082.0
2,20 a 29,Crítics,No iniciada,3,39.693746,842503.0,334421.0
3,30 a 39,Crítics,Completa,5,67.276562,991427.0,666998.0
4,30 a 39,Crítics,No iniciada,20,32.723438,991427.0,324429.0
5,40 a 49,Crítics,Completa,17,71.095045,1296148.0,921497.0
6,40 a 49,Crítics,No iniciada,40,28.904955,1296148.0,374651.0
7,50 a 59,Crítics,Completa,29,82.032362,1104675.0,906191.0
8,50 a 59,Crítics,No iniciada,48,17.967638,1104675.0,198484.0
9,60 a 69,Crítics,Completa,88,84.617303,850137.0,719363.0


In [111]:
df_uci_norm["RISC"] = df_uci_norm["RECOMPTE"]/df_uci_norm["TOTAL_GRUP"]

In [112]:
df_uci_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,0 a 19,Crítics,No iniciada,5,70.318031,1571358.0,1104948.0,0.000005
1,20 a 29,Crítics,Completa,1,60.306254,842503.0,508082.0,0.000002
2,20 a 29,Crítics,No iniciada,3,39.693746,842503.0,334421.0,0.000009
3,30 a 39,Crítics,Completa,5,67.276562,991427.0,666998.0,0.000007
4,30 a 39,Crítics,No iniciada,20,32.723438,991427.0,324429.0,0.000062
5,40 a 49,Crítics,Completa,17,71.095045,1296148.0,921497.0,0.000018
6,40 a 49,Crítics,No iniciada,40,28.904955,1296148.0,374651.0,0.000107
7,50 a 59,Crítics,Completa,29,82.032362,1104675.0,906191.0,0.000032
8,50 a 59,Crítics,No iniciada,48,17.967638,1104675.0,198484.0,0.000242
9,60 a 69,Crítics,Completa,88,84.617303,850137.0,719363.0,0.000122


In [113]:
df_uci_norm.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afect_vac_uci_brut.xlsx", sheet_name='Fulla1')

In [114]:
# select pair rows (no vacunat)
df_uci_norm_no_vac = df_uci_norm.iloc[::2]
df_uci_norm_no_vac

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,0 a 19,Crítics,No iniciada,5,70.318031,1571358.0,1104948.0,0.000005
2,20 a 29,Crítics,No iniciada,3,39.693746,842503.0,334421.0,0.000009
4,30 a 39,Crítics,No iniciada,20,32.723438,991427.0,324429.0,0.000062
6,40 a 49,Crítics,No iniciada,40,28.904955,1296148.0,374651.0,0.000107
8,50 a 59,Crítics,No iniciada,48,17.967638,1104675.0,198484.0,0.000242
10,60 a 69,Crítics,No iniciada,58,15.382697,850137.0,130774.0,0.000444
12,70 a 79,Crítics,No iniciada,29,7.900045,632085.0,49935.0,0.000581
14,80 o més,Crítics,No iniciada,6,5.395138,451425.0,24355.0,0.000246


In [115]:
# select odd rows
df_uci_norm_completa = df_uci_norm.iloc[1:-1:2] # odd rows
df_uci_norm_completa

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
1,20 a 29,Crítics,Completa,1,60.306254,842503.0,508082.0,0.000002
3,30 a 39,Crítics,Completa,5,67.276562,991427.0,666998.0,0.000007
5,40 a 49,Crítics,Completa,17,71.095045,1296148.0,921497.0,0.000018
7,50 a 59,Crítics,Completa,29,82.032362,1104675.0,906191.0,0.000032
9,60 a 69,Crítics,Completa,88,84.617303,850137.0,719363.0,0.000122
11,70 a 79,Crítics,Completa,90,92.099955,632085.0,582150.0,0.000155
13,80 o més,Crítics,Completa,49,94.604862,451425.0,427070.0,0.000115


In [116]:
# delete non necessary fields 
df_uci_norm_no_vac_rf = df_uci_norm_no_vac.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_uci_norm_no_vac_rf

,EDAT,RISC
0,0 a 19,0.000005
2,20 a 29,0.000009
4,30 a 39,0.000062
6,40 a 49,0.000107
8,50 a 59,0.000242
10,60 a 69,0.000444
12,70 a 79,0.000581
14,80 o més,0.000246


In [117]:
# delete non necessary fields 
df_uci_norm_completa_rf = df_uci_norm_completa.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_uci_norm_completa_rf

,EDAT,RISC
1,20 a 29,0.000002
3,30 a 39,0.000007
5,40 a 49,0.000018
7,50 a 59,0.000032
9,60 a 69,0.000122
11,70 a 79,0.000155
13,80 o més,0.000115


In [118]:
df_uci_norm_no_vac_rf_ren = df_uci_norm_no_vac_rf.rename(columns={"RISC": "RISC_N_V",})

In [119]:
 df_calc_uci_m = pd.merge(df_uci_norm_no_vac_rf_ren, df_uci_norm_completa_rf, how='left')

In [120]:
df_calc_uci_m

,EDAT,RISC_N_V,RISC
0,0 a 19,0.000005,NaN
1,20 a 29,0.000009,0.000002
2,30 a 39,0.000062,0.000007
3,40 a 49,0.000107,0.000018
4,50 a 59,0.000242,0.000032
5,60 a 69,0.000444,0.000122
6,70 a 79,0.000581,0.000155
7,80 o més,0.000246,0.000115


In [121]:
df_calc_uci_m['RISC'] = df_calc_uci_m['RISC'].fillna(0)

In [122]:
# efectivitat = (risc placebo - risc vacunats) / risc placebo
df_calc_uci_m['EFECTIVITAT'] = (df_calc_uci_m['RISC_N_V'] - df_calc_uci_m['RISC']) / df_calc_uci_m['RISC_N_V'] * 100

In [123]:
df_calc_uci_m

,EDAT,RISC_N_V,RISC,EFECTIVITAT
0,0 a 19,0.000005,0.000000,100.000000
1,20 a 29,0.000009,0.000002,78.059906
2,30 a 39,0.000062,0.000007,87.839956
3,40 a 49,0.000107,0.000018,82.720869
4,50 a 59,0.000242,0.000032,86.766872
5,60 a 69,0.000444,0.000122,72.417858
6,70 a 79,0.000581,0.000155,73.379595
7,80 o més,0.000246,0.000115,53.427034


In [124]:
df_calc_uci_m.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afectivitat_vacuna_critics_uci.xlsx", sheet_name='Fulla1')

En la franja de 0 a 19 al comptar amb 5 pacients crítics i cap d'ell vacunat dóna una efectivitat del 100%.

## 4. Com afecta la vacunació per a la prevenció de defuncions?

In [125]:
df_defuncions = df_prova_casos_compl[df_prova_casos_compl.ESDEVENIMENT == 'Defunció']

In [126]:
df_defuncions

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
4,0 a 19,Defunció,No iniciada,1
28,40 a 49,Defunció,Completa,3
29,40 a 49,Defunció,No iniciada,2
38,50 a 59,Defunció,Completa,4
39,50 a 59,Defunció,No iniciada,3
49,60 a 69,Defunció,Completa,16
50,60 a 69,Defunció,No iniciada,11
60,70 a 79,Defunció,Completa,40
61,70 a 79,Defunció,No iniciada,8
71,80 o més,Defunció,Completa,136


In [127]:
df_vac_dates_edat__comp

,EDAT,DOSI,RECOMPTE,Total,PERCENT
1,0 a 19,2,466410,1571358.0,29.681969
4,20 a 29,2,508082,842503.0,60.306254
7,30 a 39,2,666998,991427.0,67.276562
10,40 a 49,2,921497,1296148.0,71.095045
13,50 a 59,2,906191,1104675.0,82.032362
16,60 a 69,2,719363,850137.0,84.617303
19,70 a 79,2,582150,632085.0,92.099955
22,80 o més,2,427070,451425.0,94.604862
25,No classificat,2,25365,NaN,NaN


In [128]:
df_defuncions_norm = pd.merge(df_defuncions, df_vac_dates_edat__comp[['EDAT', 'PERCENT', 'Total']], how='left')

In [129]:
df_defuncions_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total
0,0 a 19,Defunció,No iniciada,1,29.681969,1571358.0
1,40 a 49,Defunció,Completa,3,71.095045,1296148.0
2,40 a 49,Defunció,No iniciada,2,71.095045,1296148.0
3,50 a 59,Defunció,Completa,4,82.032362,1104675.0
4,50 a 59,Defunció,No iniciada,3,82.032362,1104675.0
5,60 a 69,Defunció,Completa,16,84.617303,850137.0
6,60 a 69,Defunció,No iniciada,11,84.617303,850137.0
7,70 a 79,Defunció,Completa,40,92.099955,632085.0
8,70 a 79,Defunció,No iniciada,8,92.099955,632085.0
9,80 o més,Defunció,Completa,136,94.604862,451425.0


In [130]:
df_defuncions_norm = df_defuncions_norm.drop([0]).reset_index().drop(['index'], axis=1)

In [131]:
# df_defuncions_norm = df_defuncions_norm.drop([0])

In [132]:
# % * -1 + 100 = % no vac
df_defuncions_norm.loc[df_defuncions_norm.PAUTA == 'No iniciada', "PERCENT"] = df_uci_norm['PERCENT']*-1+100

In [133]:
df_defuncions_norm['TOTAL_GRUP'] = df_defuncions_norm['Total']*(df_defuncions_norm['PERCENT']/100)

In [134]:
df_defuncions_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP
0,40 a 49,Defunció,Completa,3,71.095045,1296148.0,921497.000000
1,40 a 49,Defunció,No iniciada,2,39.693746,1296148.0,514489.693577
2,50 a 59,Defunció,Completa,4,82.032362,1104675.0,906191.000000
3,50 a 59,Defunció,No iniciada,3,32.723438,1104675.0,361487.639105
4,60 a 69,Defunció,Completa,16,84.617303,850137.0,719363.000000
5,60 a 69,Defunció,No iniciada,11,28.904955,850137.0,245731.719824
6,70 a 79,Defunció,Completa,40,92.099955,632085.0,582150.000000
7,70 a 79,Defunció,No iniciada,8,17.967638,632085.0,113570.741748
8,80 o més,Defunció,Completa,136,94.604862,451425.0,427070.000000
9,80 o més,Defunció,No iniciada,24,15.382697,451425.0,69441.340572


In [135]:
df_defuncions_norm["RISC"] = df_defuncions_norm["RECOMPTE"]/df_defuncions_norm["TOTAL_GRUP"]

In [136]:
df_defuncions_norm

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,40 a 49,Defunció,Completa,3,71.095045,1296148.0,921497.000000,0.000003
1,40 a 49,Defunció,No iniciada,2,39.693746,1296148.0,514489.693577,0.000004
2,50 a 59,Defunció,Completa,4,82.032362,1104675.0,906191.000000,0.000004
3,50 a 59,Defunció,No iniciada,3,32.723438,1104675.0,361487.639105,0.000008
4,60 a 69,Defunció,Completa,16,84.617303,850137.0,719363.000000,0.000022
5,60 a 69,Defunció,No iniciada,11,28.904955,850137.0,245731.719824,0.000045
6,70 a 79,Defunció,Completa,40,92.099955,632085.0,582150.000000,0.000069
7,70 a 79,Defunció,No iniciada,8,17.967638,632085.0,113570.741748,0.000070
8,80 o més,Defunció,Completa,136,94.604862,451425.0,427070.000000,0.000318
9,80 o més,Defunció,No iniciada,24,15.382697,451425.0,69441.340572,0.000346


In [137]:
df_defuncions_norm.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afect_vac_defuncions_brut.xlsx", sheet_name='Fulla1')

Donats els resultats previs calcularem l'afectivitat per als rangs d'edat entre 40 fins a 80 o més.

In [138]:
# select pair rows (vacunat)
df_defuncions_norm_completa = df_defuncions_norm.iloc[::2]
df_defuncions_norm_completa

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
0,40 a 49,Defunció,Completa,3,71.095045,1296148.0,921497.0,0.000003
2,50 a 59,Defunció,Completa,4,82.032362,1104675.0,906191.0,0.000004
4,60 a 69,Defunció,Completa,16,84.617303,850137.0,719363.0,0.000022
6,70 a 79,Defunció,Completa,40,92.099955,632085.0,582150.0,0.000069
8,80 o més,Defunció,Completa,136,94.604862,451425.0,427070.0,0.000318


In [139]:
# select odd rows
df_defuncions_norm_no_vac = df_defuncions_norm.iloc[1:10:2] # odd rows, 10 = 9 +1 del index 0 
df_defuncions_norm_no_vac

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE,PERCENT,Total,TOTAL_GRUP,RISC
1,40 a 49,Defunció,No iniciada,2,39.693746,1296148.0,514489.693577,0.000004
3,50 a 59,Defunció,No iniciada,3,32.723438,1104675.0,361487.639105,0.000008
5,60 a 69,Defunció,No iniciada,11,28.904955,850137.0,245731.719824,0.000045
7,70 a 79,Defunció,No iniciada,8,17.967638,632085.0,113570.741748,0.000070
9,80 o més,Defunció,No iniciada,24,15.382697,451425.0,69441.340572,0.000346


In [140]:
# delete non necessary fields 
df_defuncions_norm_no_vac_rf = df_defuncions_norm_no_vac.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_defuncions_norm_no_vac_rf

,EDAT,RISC
1,40 a 49,0.000004
3,50 a 59,0.000008
5,60 a 69,0.000045
7,70 a 79,0.000070
9,80 o més,0.000346


In [141]:
# delete non necessary fields 
df_defuncions_norm_completa_rf = df_defuncions_norm_completa.drop(columns=['ESDEVENIMENT', 'PAUTA', 'RECOMPTE', 'PERCENT', 'Total', 'TOTAL_GRUP'])
df_defuncions_norm_completa_rf

,EDAT,RISC
0,40 a 49,0.000003
2,50 a 59,0.000004
4,60 a 69,0.000022
6,70 a 79,0.000069
8,80 o més,0.000318


In [142]:
df_defuncions_norm_no_vac_rf_ren = df_defuncions_norm_no_vac_rf.rename(columns={"RISC": "RISC_N_V",})

In [143]:
df_defuncions_norm_no_vac_rf_ren

,EDAT,RISC_N_V
1,40 a 49,0.000004
3,50 a 59,0.000008
5,60 a 69,0.000045
7,70 a 79,0.000070
9,80 o més,0.000346


In [144]:
df_calc_defuncions_m = pd.merge(df_defuncions_norm_no_vac_rf_ren, df_defuncions_norm_completa_rf, how='right')

In [145]:
df_calc_defuncions_m = df_defuncions_norm_no_vac_rf_ren.merge(df_defuncions_norm_completa_rf, left_on='EDAT', right_on='EDAT')

In [146]:
df_calc_defuncions_m

,EDAT,RISC_N_V,RISC
0,40 a 49,0.000004,0.000003
1,50 a 59,0.000008,0.000004
2,60 a 69,0.000045,0.000022
3,70 a 79,0.000070,0.000069
4,80 o més,0.000346,0.000318


In [147]:
# efectivitat = (risc placebo - risc vacunats) / risc placebo
df_calc_defuncions_m['EFECTIVITAT'] = (df_calc_defuncions_m['RISC_N_V'] - df_calc_defuncions_m['RISC']) / df_calc_defuncions_m['RISC_N_V'] * 100

In [148]:
df_calc_defuncions_m

,EDAT,RISC_N_V,RISC,EFECTIVITAT
0,40 a 49,0.000004,0.000003,16.252083
1,50 a 59,0.000008,0.000004,46.812149
2,60 a 69,0.000045,0.000022,50.313269
3,70 a 79,0.000070,0.000069,2.455775
4,80 o més,0.000346,0.000318,7.860320


In [149]:
df_calc_defuncions_m.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/afectivitat_vacuna_defuncions.xlsx", sheet_name='Fulla1')

Tot i mostrar una afectació positiva observem com per les franges de més edat l'afectació és menor que en els ingressos a UCI. Cal tenir en compte que algunes de les franges d'edat tenen un nombre de defuncions poc elevat la probabilitat d'error estadístic és elevada.

Cal tenir present que la quantitat de casos abans del 10 d'Octubre és semblant a les següents però que les dues primeres setmanes de desembre va aver-hi un augment dels casos influenciats per l'arribada d'Omicron que no es veuen reflectides en les defuncions perqè la mitjana entre la detecció del cas i la defunció pot anar de dies a mesos i part de les defuncions del període d'estudi poden haver estat detectats molt abans de la defunció. En canvi l'entrada a UCI és molt més propera a la infecció.

## 5.Càlcul de la probabilitat de defunció segons número de casos en el mateix període.

En la resposta d'aquesta pregunta caldrà tenir present les conclusions de l'exercici 4.

In [150]:
# we choose only the number of cases from EDEVENIMENT
df_prova_casos_compl_casos = df_prova_casos_compl[df_prova_casos_compl.ESDEVENIMENT == 'Cas']

In [151]:
df_prova_casos_compl_casos

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
0,0 a 19,Cas,Completa,1912
1,0 a 19,Cas,No iniciada,16306
7,20 a 29,Cas,Completa,3976
8,20 a 29,Cas,No iniciada,1911
15,30 a 39,Cas,Completa,7074
16,30 a 39,Cas,No iniciada,2683
22,40 a 49,Cas,Completa,11866
23,40 a 49,Cas,No iniciada,2505
33,50 a 59,Cas,Completa,7381
34,50 a 59,Cas,No iniciada,1314


In [154]:
df_defuncions = df_prova_casos_compl[df_prova_casos_compl.ESDEVENIMENT == 'Defunció']

In [155]:
df_defuncions

,EDAT,ESDEVENIMENT,PAUTA,RECOMPTE
4,0 a 19,Defunció,No iniciada,1
28,40 a 49,Defunció,Completa,3
29,40 a 49,Defunció,No iniciada,2
38,50 a 59,Defunció,Completa,4
39,50 a 59,Defunció,No iniciada,3
49,60 a 69,Defunció,Completa,16
50,60 a 69,Defunció,No iniciada,11
60,70 a 79,Defunció,Completa,40
61,70 a 79,Defunció,No iniciada,8
71,80 o més,Defunció,Completa,136


In [156]:
df_calc_prob_def = pd.merge(df_prova_casos_compl_casos, df_defuncions,  how='left', left_on=['EDAT','PAUTA'], right_on = ['EDAT','PAUTA'])

In [157]:
df_calc_prob_def

,EDAT,ESDEVENIMENT_x,PAUTA,RECOMPTE_x,ESDEVENIMENT_y,RECOMPTE_y
0,0 a 19,Cas,Completa,1912,NaN,NaN
1,0 a 19,Cas,No iniciada,16306,Defunció,1.0
2,20 a 29,Cas,Completa,3976,NaN,NaN
3,20 a 29,Cas,No iniciada,1911,NaN,NaN
4,30 a 39,Cas,Completa,7074,NaN,NaN
5,30 a 39,Cas,No iniciada,2683,NaN,NaN
6,40 a 49,Cas,Completa,11866,Defunció,3.0
7,40 a 49,Cas,No iniciada,2505,Defunció,2.0
8,50 a 59,Cas,Completa,7381,Defunció,4.0
9,50 a 59,Cas,No iniciada,1314,Defunció,3.0


In [158]:
df_calc_prob_def = df_calc_prob_def.fillna(0)

In [159]:
df_calc_prob_def

,EDAT,ESDEVENIMENT_x,PAUTA,RECOMPTE_x,ESDEVENIMENT_y,RECOMPTE_y
0,0 a 19,Cas,Completa,1912,0,0.0
1,0 a 19,Cas,No iniciada,16306,Defunció,1.0
2,20 a 29,Cas,Completa,3976,0,0.0
3,20 a 29,Cas,No iniciada,1911,0,0.0
4,30 a 39,Cas,Completa,7074,0,0.0
5,30 a 39,Cas,No iniciada,2683,0,0.0
6,40 a 49,Cas,Completa,11866,Defunció,3.0
7,40 a 49,Cas,No iniciada,2505,Defunció,2.0
8,50 a 59,Cas,Completa,7381,Defunció,4.0
9,50 a 59,Cas,No iniciada,1314,Defunció,3.0


In [165]:
# create a new row with total sum of other rows
df_calc_prob_def_sum = df_calc_prob_def.append(df_calc_prob_def.sum(numeric_only=True), ignore_index=True)

In [166]:
df_calc_prob_def_sum

,EDAT,ESDEVENIMENT_x,PAUTA,RECOMPTE_x,ESDEVENIMENT_y,RECOMPTE_y
0,0 a 19,Cas,Completa,1912.0,0,0.0
1,0 a 19,Cas,No iniciada,16306.0,Defunció,1.0
2,20 a 29,Cas,Completa,3976.0,0,0.0
3,20 a 29,Cas,No iniciada,1911.0,0,0.0
4,30 a 39,Cas,Completa,7074.0,0,0.0
5,30 a 39,Cas,No iniciada,2683.0,0,0.0
6,40 a 49,Cas,Completa,11866.0,Defunció,3.0
7,40 a 49,Cas,No iniciada,2505.0,Defunció,2.0
8,50 a 59,Cas,Completa,7381.0,Defunció,4.0
9,50 a 59,Cas,No iniciada,1314.0,Defunció,3.0


In [167]:
df_calc_prob_def_sum['PROB_DEF'] = df_calc_prob_def_sum['RECOMPTE_y'] / df_calc_prob_def_sum['RECOMPTE_x'] 

In [168]:
df_calc_prob_def_sum

,EDAT,ESDEVENIMENT_x,PAUTA,RECOMPTE_x,ESDEVENIMENT_y,RECOMPTE_y,PROB_DEF
0,0 a 19,Cas,Completa,1912.0,0,0.0,0.000000
1,0 a 19,Cas,No iniciada,16306.0,Defunció,1.0,0.000061
2,20 a 29,Cas,Completa,3976.0,0,0.0,0.000000
3,20 a 29,Cas,No iniciada,1911.0,0,0.0,0.000000
4,30 a 39,Cas,Completa,7074.0,0,0.0,0.000000
5,30 a 39,Cas,No iniciada,2683.0,0,0.0,0.000000
6,40 a 49,Cas,Completa,11866.0,Defunció,3.0,0.000253
7,40 a 49,Cas,No iniciada,2505.0,Defunció,2.0,0.000798
8,50 a 59,Cas,Completa,7381.0,Defunció,4.0,0.000542
9,50 a 59,Cas,No iniciada,1314.0,Defunció,3.0,0.002283


In [169]:
df_calc_prob_def_sum.to_excel("C:/Users/34639/wa/impacte_covid_per_estat_vacunacio/resultats/prob_defuncio.xlsx", sheet_name='Fulla1')

Tot i que per interpretar aquesta dada s'ha de tenir present el que s'ha mencionat abans es veu clarament dos dels factors de risc de defunció per coronavirus, la vacunació i l'edat. També cal tindre present que la variant majoritària durant el perídode d'estudi i les setmanes anteriors era la Delta.